In [ ]:
# DQN original con target network y replay buffer

import torch
import torch.nn as nn
import torch.optim as optim
import gymnasium as gym
import random
import numpy as np
import collections # Para el Replay Buffer (deque)
import matplotlib.pyplot as plt # Para graficar

plt.rcParams.update({
    'figure.facecolor': '#2b2b2b',     # fondo exterior
    'axes.facecolor':   '#2b2b2b',     # fondo del área del gráfico
    'axes.edgecolor':   '#cccccc',     # borde de los ejes
    'axes.labelcolor':  '#cccccc',     # etiquetas de los ejes
    'xtick.color':      '#cccccc',     # color de los números en el eje X
    'ytick.color':      '#cccccc',     # color de los números en el eje Y
    'text.color':       '#cccccc',     # color del título y textos
    'grid.color':       '#444444',     # color de la grilla
})

# --- Hiperparámetros ---
#max_episodes = 500        # Número de episodios a entrenar
#max_steps_per_episode = 500 # Límite de pasos por episodio (importante en CartPole)
#learning_rate = 0.001     # Tasa de aprendizaje
#gamma = 0.99              # Factor de descuento
#epsilon_start = 1.0       # Epsilon inicial (exploración)
#epsilon_end = 0.01        # Epsilon final
#epsilon_decay_episodes = 300 # Episodios durante los cuales decae epsilon
#buffer_size = 10000       # Tamaño del Replay Buffer
#batch_size = 64           # Tamaño del lote para aprender del buffer
#target_update_freq = 100  # Frecuencia (en steps) para actualizar la Target Network
#print_every = 50          # Imprimir progreso cada N episodios
#smoothing_window = 50     # Ventana para suavizar la curva de recompensa

max_episodes = 1000        # Número de episodios a entrenar
max_steps_per_episode = 500 # Límite de pasos por episodio (importante en CartPole)
learning_rate = 0.001     # Tasa de aprendizaje
gamma = 0.99              # Factor de descuento
epsilon_start = 1.0       # Epsilon inicial (exploración)
epsilon_end = 0.001        # Epsilon final
epsilon_decay_episodes = 300 # Episodios durante los cuales decae epsilon
buffer_size = 10_000       # Tamaño del Replay Buffer
batch_size = 64           # Tamaño del lote para aprender del buffer
target_update_freq = 100  # Frecuencia (en steps) para actualizar la Target Network
print_every = 50          # Imprimir progreso cada N episodios
smoothing_window = 50     # Ventana para suavizar la curva de recompensa

# --- Entorno ---
env = gym.make('CartPole-v1')
n_observations = env.observation_space.shape[0]
n_actions = env.action_space.n

# --- Red Neuronal (Q-Network) ---
def create_q_network():
    return nn.Sequential(
        nn.Linear(n_observations, 128),
        nn.ReLU(),
        nn.Linear(128, 128),
        nn.ReLU(),
        nn.Linear(128, n_actions)
    )

# Crea la red principal (online) y la red objetivo (target)
q_network = create_q_network()
target_network = create_q_network()
# Copia pesos iniciales a la red objetivo
target_network.load_state_dict(q_network.state_dict())
# poner la red objetivo en modo evaluación
target_network.eval()

optimizer = optim.Adam(q_network.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

# --- Replay Buffer ---
Transition = collections.namedtuple('Transition',
                                    ('state', 'action', 'reward', 'next_state', 'done'))
replay_buffer = collections.deque(maxlen=buffer_size)

# --- Función para actualizar la Red Objetivo ---
def update_target_network():
    target_network.load_state_dict(q_network.state_dict())

# --- Entrenamiento ---
global_step = 0
epsilon = epsilon_start
episode_rewards_history = [] # <<<--- Lista para guardar recompensas por episodio

print("--- Iniciando Entrenamiento ---")
for episode in range(max_episodes):
    obs, info = env.reset()
    state = torch.tensor(obs, dtype=torch.float32).unsqueeze(0) # Añadir dim de batch
    episode_reward = 0
    episode_steps = 0

    for step in range(max_steps_per_episode):
        global_step += 1
        episode_steps += 1

        # --- Selección de Acción (Epsilon-Greedy) ---
        if random.random() < epsilon:
            action = torch.tensor([[env.action_space.sample()]], dtype=torch.long) # Acción aleatoria
        else:
            with torch.no_grad():
                q_values = q_network(state)
                action = q_values.max(1)[1].view(1, 1)

        # --- Ejecutar acción en el entorno ---
        next_obs, reward, terminated, truncated, info = env.step(action.item())
        done = terminated or truncated
        episode_reward += reward

        # Crear tensores para la transición
        reward_tensor = torch.tensor([reward], dtype=torch.float32)
        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(next_obs, dtype=torch.float32).unsqueeze(0)
        done_tensor = torch.tensor([done], dtype=torch.float32)

        # --- Guardar transición en el Replay Buffer ---
        replay_buffer.append(Transition(state, action, reward_tensor, next_state, done_tensor))

        # Mover al siguiente estado
        state = next_state

        # --- Aprendizaje desde el Replay Buffer ---
        if len(replay_buffer) >= batch_size:
            transitions = random.sample(replay_buffer, batch_size)
            batch = Transition(*zip(*transitions))

            non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), dtype=torch.bool)
            non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])

            state_batch = torch.cat(batch.state)
            action_batch = torch.cat(batch.action)
            reward_batch = torch.cat(batch.reward)

            state_action_values = q_network(state_batch).gather(1, action_batch).squeeze(1)

            next_state_values = torch.zeros(batch_size)
            with torch.no_grad():
                 next_state_values[non_final_mask] = target_network(non_final_next_states).max(1)[0]

            expected_state_action_values = reward_batch + (gamma * next_state_values)

            loss = loss_fn(state_action_values, expected_state_action_values)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # --- Actualiz. la Red Objetivo ---
        if global_step % target_update_freq == 0:
            update_target_network()

        if done:
            break # Salir del bucle de pasos si el episodio terminó

    # --- Guarda recompensa y decaimiento de Epsilon ---
    episode_rewards_history.append(episode_reward) # <<<--- Guarda recompensa
    epsilon = max(epsilon_end, epsilon_start - (episode / epsilon_decay_episodes) * (epsilon_start - epsilon_end))

    if (episode + 1) % print_every == 0:
        # Calc. recompensa promedio reciente para el print
        avg_reward = np.mean(episode_rewards_history[-print_every:])
        print(f'Episodio: {episode + 1}/{max_episodes}, Pasos: {episode_steps}, Recompensa Promedio ({print_every} ep): {avg_reward:.2f}, Epsilon: {epsilon:.3f}')

print("--- Entrenamiento Finalizado ---")

# --- Grafic. Curva de Convergencia ---
print("\n--- Generando Gráfico de Convergencia ---")
plt.figure(figsize=(12, 6))
plt.plot(range(1, max_episodes + 1), episode_rewards_history, label='Recompensa por Episodio', alpha=0.4)

# Calc. y graficar la media móvil para suavizar
if len(episode_rewards_history) >= smoothing_window:
    # se usa convolve para un cálculo eficiente de la media móvil
    rewards_smoothed = np.convolve(episode_rewards_history, np.ones(smoothing_window)/smoothing_window, mode='valid')
    # Ajuste del eje x para la curva suavizada
    plt.plot(range(smoothing_window, max_episodes + 1), rewards_smoothed, label=f'Media Móvil ({smoothing_window} episodios)', color='red', linewidth=2)

plt.xlabel("Episodio")
plt.ylabel("Recompensa Total")
plt.title("Convergencia de Recompensa DQN en CartPole-v1")
plt.legend()
plt.grid(True)
plt.show()


# --- (Evaluación SIN RENDERIZADO) ---
print("\n--- evaluando la política aprendida ---")
# Crea env SIN render_mode para la evaluación final
eval_env = gym.make('CartPole-v1')

total_eval_reward = 0
num_eval_episodes = 10 # Evalua durante 10 episodios

for i in range(num_eval_episodes):
    obs, info = eval_env.reset()
    state = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)
    done = False
    episode_eval_reward = 0
    episode_steps = 0
    while not done and episode_steps < max_steps_per_episode:
        # NO HAY env.render() aquí
        with torch.no_grad():
            q_values = q_network(state) # Usar la red ONLINE entrenada
            action = q_values.max(1)[1].view(1, 1) # Elegir la mejor acción (sin epsilon)

        obs, reward, terminated, truncated, info = eval_env.step(action.item())
        done = terminated or truncated
        episode_eval_reward += reward
        episode_steps += 1

        if not done:
            state = torch.tensor(obs, dtype=torch.float32).unsqueeze(0)

    print(f'Evaluación {i+1}, Recompensa: {episode_eval_reward}, Pasos: {episode_steps}')
    total_eval_reward += episode_eval_reward

print(f'\nRecompensa Promedio en Evaluación ({num_eval_episodes} episodios): {total_eval_reward / num_eval_episodes:.2f}')
eval_env.close()
env.close() # Cerrar también el entorno de entrenamiento